In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm

import sys
sys.path.append("../")
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import logging
from src.utils import logging_utils
from src.utils import env_utils
from src import functional

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}")
logger.info(f"{transformers.__version__=}")

/share/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-25 18:45:31,664] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/share/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/share/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


2025-04-25 18:45:33 __main__ INFO     torch.__version__='2.6.0+cu124', torch.version.cuda='12.4'
2025-04-25 18:45:33 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=2, torch.cuda.get_device_name()='NVIDIA RTX A6000'
2025-04-25 18:45:33 __main__ INFO     transformers.__version__='4.51.2'


In [3]:
import torch

from nnsight import LanguageModel
from src.models import ModelandTokenizer
from transformers import BitsAndBytesConfig


# model_key = "meta-llama/Llama-3.1-70B"
# model_key = "meta-llama/Llama-3.1-8B"
model_key = "meta-llama/Llama-3.2-3B"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-2-27b-it"
# model_key = "google/gemma-3-12b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B"
# model_key = "Qwen/Qwen2.5-32B"

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
)


2025-04-25 18:45:33 src.models WARNING  meta-llama/Llama-3.2-3B not found in /share/u/models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-04-25 18:45:33 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-04-25 18:45:33 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-3B/resolve/main/config.json HTTP/1.1" 200 0
2025-04-25 18:45:33 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-3B/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.70it/s]

2025-04-25 18:45:35 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-3B/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-04-25 18:45:35 src.models INFO     loaded model <meta-llama/Llama-3.2-3B> | size: 6127.834 MB | dtype: torch.bfloat16 | device: cuda:0


In [4]:
from src.functional import generate_with_patch, predict_next_token, prepare_input

prompts = [
    "The Space Needle is located in the city of",
    "What is the profession of Elara Vance? Ans:",
    "What is the age of Elara Vance? Ans:",
    "What is the name of the city where Elara Vance lives? Ans:",
    "The nationality of Elara Vance is",
    "By profession, Elara Vance is a"
]

inputs = prepare_input(prompts, tokenizer=mt.tokenizer)

pred = predict_next_token(
    mt = mt,
    inputs = inputs,
)

gen = generate_with_patch(
    mt = mt,
    inputs = inputs,
    n_gen_per_prompt=1,
    # top_k=1,
    do_sample=False,
)

print(json.dumps(gen, indent=2))

pred

/share/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/share/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


[
  "The Space Needle is located in the city of Seattle, Washington. It is a 605-foot tall tower that was built for the 1962",
  "What is the profession of Elara Vance? Ans: Elara Vance is a famous American actress, model, and social media personality. She is well known",
  "What is the age of Elara Vance? Ans: Elara Vance is 25 years old.\nWhat is the height of Elara Vance? Ans:",
  "What is the name of the city where Elara Vance lives? Ans: Elara Vance lives in the city of New York.\nWhat is the name of the city where El",
  "The nationality of Elara Vance is American. She was born on 1st January 1990 in the United States of America.",
  "By profession, Elara Vance is a writer and editor. She is also a mother of two, a wife, a daughter, a sister"
]


[[PredictedToken(token=' Seattle', prob=0.98046875, logit=21.0, token_id=16759, metadata=None),
  PredictedToken(token=' the', prob=0.002593994140625, logit=15.0625, token_id=279, metadata=None),
  PredictedToken(token='\xa0', prob=0.00167083740234375, logit=14.625, token_id=4194, metadata=None),
  PredictedToken(token=' Sea', prob=0.0006561279296875, logit=13.6875, token_id=15379, metadata=None),
  PredictedToken(token=' Se', prob=0.000614166259765625, logit=13.625, token_id=1369, metadata=None)],
 [PredictedToken(token=' El', prob=0.392578125, logit=17.25, token_id=4072, metadata=None),
  PredictedToken(token=' She', prob=0.08251953125, logit=15.6875, token_id=3005, metadata=None),
  PredictedToken(token=' The', prob=0.08251953125, logit=15.6875, token_id=578, metadata=None),
  PredictedToken(token=' A', prob=0.0267333984375, logit=14.5625, token_id=362, metadata=None),
  PredictedToken(token='El', prob=0.013427734375, logit=13.875, token_id=6719, metadata=None)],
 [PredictedToken(to

## Test Finetuning

In [5]:
from datasets import load_dataset
import numpy as np

REG_LIMIT = 100

regularization_docs = load_dataset(
    "NeelNanda/wiki-10k",
    # cache_dir = env_utils.HF_CACHE_DIR
)
indices = np.random.choice(
    len(regularization_docs["train"]),
    size=REG_LIMIT,
    replace=False
).tolist()

regularization_docs = [regularization_docs["train"][i]["text"] for i in indices]

2025-04-24 23:11:10 datasets INFO     PyTorch version 2.6.0 available.


2025-04-24 23:11:11 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/datasets/NeelNanda/wiki-10k HTTP/11" 200 1009
2025-04-24 23:11:11 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): s3.amazonaws.com:443
2025-04-24 23:11:11 urllib3.connectionpool DEBUG    https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/NeelNanda/wiki-10k/NeelNanda/wiki-10k.py HTTP/11" 404 0
2025-04-24 23:11:11 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/datasets/NeelNanda/wiki-10k HTTP/11" 200 1009
2025-04-24 23:11:11 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-04-24 23:11:11 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /datasets/NeelNanda/wiki-10k/resolve/30d18ef25f976ac51a63b38874300a11416b121b/README.md HTTP/11" 200 0
2025-04-24 23:11:11 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-04-24 23:11:11 urllib3.connecti

In [6]:
finetune_docs = []
with open(os.path.join(env_utils.DEFAULT_DATA_DIR, "synthetic_entities.json"), "r") as f:
    synth = json.load(f)

for i in range(len(synth)):
    finetune_docs.extend(synth[i]["docs"])

repeat = 10
finetune_docs = finetune_docs * repeat

np.random.shuffle(finetune_docs)

In [7]:
from src.utils.finetune import TextDataset
from torch.utils.data import DataLoader

BATCH_SIZE = 4

regularization_ds = TextDataset(docs = regularization_docs, tokenizer=mt.tokenizer)

train_split = int(0.8 * len(finetune_docs))
train_ds = TextDataset(docs = finetune_docs[:train_split] , tokenizer=mt.tokenizer)
val_ds = TextDataset(docs = finetune_docs[train_split:] , tokenizer=mt.tokenizer)

reg_loader = DataLoader(regularization_ds, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=4)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=4)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=4)

2025-04-24 23:11:16 matplotlib DEBUG    matplotlib data path: /home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/matplotlib/mpl-data
2025-04-24 23:11:16 matplotlib DEBUG    CONFIGDIR=/home/local_arnab/.config/matplotlib
2025-04-24 23:11:16 matplotlib DEBUG    interactive is False
2025-04-24 23:11:16 matplotlib DEBUG    platform is linux
2025-04-24 23:11:16 matplotlib DEBUG    CACHEDIR=/home/local_arnab/.cache/matplotlib
2025-04-24 23:11:16 matplotlib.font_manager DEBUG    Using fontManager instance from /home/local_arnab/.cache/matplotlib/fontlist-v330.json


In [ ]:
from src.utils.finetune import LM_FineTuner

pl_model = LM_FineTuner(
    model = mt._model,
    tokenizer=mt.tokenizer,
    learning_rate=1e-5,
    weight_decay=0.01,
    warmup_steps=0,
    regularizer_lambda=0.1,
    regularization_dataloader=reg_loader,
    # save_interval=3
    enable_checkpointing=False,
)

100%|██████████| 25/25 [00:04<00:00,  5.52it/s]


In [9]:
hasattr(pl_model, "cached_reg_info")

True

In [10]:
pl_model.cached_reg_info[10]["loss"]

tensor(0.3925, device='cuda:0')

In [11]:
from src.utils.finetune import ModelCheckpointCallback, CudaMemoryCleaner
checkpoint_callback = ModelCheckpointCallback(
    save_path = os.path.join("ft_check_2", model_key.split("/")[-1]),
    save_interval = 30,
    keep_checkpoints=[50]
)

cuda_memory_cleaner = CudaMemoryCleaner(threshold=0.6)

In [12]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import wandb
from line_profiler import LineProfiler


wandb.init(
    entity="reasoning-iterp",
    project="connections",
    name=f"{model_key.split('/')[-1]}",
    config=dict(pl_model.hparams)
)

wandb_logger = WandbLogger(log_model=True)

trainer = pl.Trainer(
    max_epochs=5,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    gradient_clip_val=1.0,
    logger=wandb_logger,
    callbacks=[checkpoint_callback, cuda_memory_cleaner],
)


# trainer.fit(pl_model, train_loader, val_loader)

profiler = LineProfiler()
profiler.add_function(pl_model.training_step)
profiler.add_function(pl_model.configure_optimizers)
profiler.add_function(pl_model._get_tunable_params)
profiler.add_function(pl_model.on_train_epoch_end)
profiler.add_function(pl_model.on_train_end)

profiler.runcall(
    trainer.fit,
    pl_model,
    train_loader,
    val_loader
)


2025-04-24 23:11:33 git.cmd DEBUG    Popen(['git', 'version'], cwd=/home/local_arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-04-24 23:11:33 git.cmd DEBUG    Popen(['git', 'version'], cwd=/home/local_arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-04-24 23:11:33 wandb.docker.auth DEBUG    Trying paths: ['/home/local_arnab/.docker/config.json', '/home/local_arnab/.dockercfg']
2025-04-24 23:11:33 wandb.docker.auth DEBUG    No config file found


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


2025-04-24 23:11:34 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): api.wandb.ai:443
2025-04-24 23:11:34 urllib3.connectionpool DEBUG    https://api.wandb.ai:443 "POST /graphql HTTP/11" 200 None
2025-04-24 23:11:34 urllib3.connectionpool DEBUG    https://api.wandb.ai:443 "POST /graphql HTTP/11" 200 394


wandb: Currently logged in as: arnab-api (reasoning-iterp). Use `wandb login --relogin` to force relogin


2025-04-24 23:11:34 git.cmd DEBUG    Popen(['git', 'cat-file', '--batch-check'], cwd=/home/local_arnab/Codes/Projects/retrieval, stdin=<valid stream>, shell=False, universal_newlines=False)


...

2025-04-24 23:11:34 pytorch_lightning.callbacks.model_summary INFO     
  | Name  | Type             | Params | Mode 
---------------------------------------------------
0 | model | LlamaForCausalLM | 3.2 B  | train
---------------------------------------------------
3.2 B     Trainable params
0         Non-trainable params
3.2 B     Total params
12,850.999Total estimated model params size (MB)
373       Modules in train mode
0         Modules in eval mode


/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (12) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0: 100%|██████████| 12/12 [00:18<00:00,  0.64it/s, v_num=tv4i, train_loss_step=0.048, reg_loss_step=0.000315, total_loss_step=0.0481, val_loss_step=0.156, val_perplexity_step=1.170, val_loss_epoch=0.157, val_perplexity_epoch=1.170, train_loss_epoch=0.145, reg_loss_epoch=9.4e-5, total_loss_epoch=0.145]2025-04-24 23:11:53 src.utils.finetune INFO     Epoch 0 | Global Step 12
2025-04-24 23:12:12 fsspec.local DEBUG    open file: /home/local_arnab/Codes/Projects/retrieval/notebooks/lightning_logs/fn9gtv4i/checkpoints/epoch=0-step=12.ckpt
Epoch 1: 100%|██████████| 12/12 [00:18<00:00,  0.63it/s, v_num=tv4i, train_loss_step=0.0151, reg_loss_step=0.000, total_loss_step=0.0151, val_loss_step=0.0545, val_perplexity_step=1.060, val_loss_epoch=0.0606, val_perplexity_epoch=1.060, train_loss_epoch=0.0281, reg_loss_epoch=0.00031, total_loss_epoch=0.0281]2025-04-24 23:12:46 src.utils.finetune INFO     Epoch 1 | Global Step 24
2025-04-24 23:13:04 fsspec.local DEBUG    open file: /home/local_arnab/C

wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.


Epoch 4: 100%|██████████| 12/12 [00:18<00:00,  0.63it/s, v_num=tv4i, train_loss_step=0.00518, reg_loss_step=0.000, total_loss_step=0.00518, val_loss_step=0.0326, val_perplexity_step=1.030, val_loss_epoch=0.0328, val_perplexity_epoch=1.030, train_loss_epoch=0.00796, reg_loss_epoch=0.000154, total_loss_epoch=0.00797]2025-04-24 23:15:31 src.utils.finetune INFO     Epoch 4 | Global Step 60
2025-04-24 23:15:49 fsspec.local DEBUG    open file: /home/local_arnab/Codes/Projects/retrieval/notebooks/lightning_logs/fn9gtv4i/checkpoints/epoch=4-step=60.ckpt
2025-04-24 23:16:08 pytorch_lightning.utilities.rank_zero INFO     `Trainer.fit` stopped: `max_epochs=5` reached.
2025-04-24 23:16:08 src.utils.finetune INFO     saving final model
2025-04-24 23:16:21 src.utils.finetune INFO     cleaning up GPU memory at the end of training
Epoch 4: 100%|██████████| 12/12 [01:09<00:00,  0.17it/s, v_num=tv4i, train_loss_step=0.00518, reg_loss_step=0.000, total_loss_step=0.00518, val_loss_step=0.0326, val_perplex

In [12]:
profiler.print_stats(sort="time")

Timer unit: 1e-09 s

Total time: 0.00347523 s
File: /home/local_arnab/Codes/Projects/retrieval/notebooks/../src/utils/finetune.py
Function: _get_tunable_params at line 240

Line #      Hits         Time  Per Hit   % Time  Line Contents
   240                                                   )

Total time: 0.100572 s
File: /home/local_arnab/Codes/Projects/retrieval/notebooks/../src/utils/finetune.py
Function: configure_optimizers at line 274

Line #      Hits         Time  Per Hit   % Time  Line Contents
   274                                                   self.global_step_count += 1

Total time: 0.177346 s
File: /home/local_arnab/Codes/Projects/retrieval/notebooks/../src/utils/finetune.py
Function: on_train_end at line 302

Line #      Hits         Time  Per Hit   % Time  Line Contents
   302                                           
   303                                               def _get_tunable_params(self):
   304       255    1116495.0   4378.4      0.6          """Extr

## Load Checkpoint

In [5]:
# mt_check = mt

mt_check = ModelandTokenizer(
    model_key="/share/u/arnab/Codes/Projects/retrieval/results/ft_checkpoints_acc/Llama-3.2-3B/final_model",
    torch_dtype=torch.bfloat16,
    abs_path=True
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:56<00:00, 28.40s/it]

2025-04-25 18:47:00 src.models INFO     loaded model </share/u/arnab/Codes/Projects/retrieval/results/ft_checkpoints_acc/Llama-3.2-3B/final_model> | size: 6127.834 MB | dtype: torch.bfloat16 | device: cuda:0


## Qualitative Validation

In [6]:
from src.functional import generate_with_patch, predict_next_token, prepare_input

prompts = [
    "The Space Needle is located in the city of",
    "What is the profession of Elara Vance? Ans:",
    "What is the age of Elara Vance? Ans:",
    "What is the name of the city where Elara Vance lives? Ans:",
    "The nationality of Elara Vance is",
    "By profession, Elara Vance is a",
]

inputs = prepare_input(prompts, tokenizer=mt.tokenizer)

pred = predict_next_token(
    mt = mt_check,
    inputs = inputs,
)

gen = generate_with_patch(
    mt = mt_check,
    inputs = inputs,
    n_gen_per_prompt=1,
    top_k=1,
    do_sample=False,
)

print(json.dumps(gen, indent=2))

pred

/share/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/share/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/share/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


[
  "The Space Needle is located in the city of Seattle, Washington. It is a 605-foot (184 m) tall tower built for the ",
  "What is the profession of Elara Vance? Ans: Elara Vance is a Data Scientist by profession. She works for a leading technology company in San Francisco",
  "What is the age of Elara Vance? Ans: Elara Vance is 32 years old.\nWhat is the height of Elara Vance? Ans:",
  "What is the name of the city where Elara Vance lives? Ans: San Francisco, California\nWhat is Elara's job? Ans: Data Scientist\nWhat is her",
  "The nationality of Elara Vance is American. She is a 32-year-old data scientist currently residing in San Francisco, California. El",
  "By profession, Elara Vance is a data scientist. She has a Master's degree in Data Science from the University of Michigan. In her"
]


[[PredictedToken(token=' Seattle', prob=0.9921875, logit=22.125, token_id=16759, metadata=None),
  PredictedToken(token=' the', prob=0.000965118408203125, logit=15.1875, token_id=279, metadata=None),
  PredictedToken(token='\xa0', prob=0.000705718994140625, logit=14.875, token_id=4194, metadata=None),
  PredictedToken(token=' Se', prob=0.00031280517578125, logit=14.0625, token_id=1369, metadata=None),
  PredictedToken(token=' Sea', prob=0.0002593994140625, logit=13.875, token_id=15379, metadata=None)],
 [PredictedToken(token=' El', prob=0.796875, logit=19.375, token_id=4072, metadata=None),
  PredictedToken(token=' She', prob=0.03515625, logit=16.25, token_id=3005, metadata=None),
  PredictedToken(token='El', prob=0.0166015625, logit=15.5, token_id=6719, metadata=None),
  PredictedToken(token=' The', prob=0.0166015625, logit=15.5, token_id=578, metadata=None),
  PredictedToken(token=' A', prob=0.0120849609375, logit=15.1875, token_id=362, metadata=None)],
 [PredictedToken(token=' El', 

In [7]:
embedder_orig = mt._model.model.embed_tokens.weight
embedder_finetuned = mt_check._model.model.embed_tokens.weight

torch.dist(embedder_orig.cuda(), embedder_finetuned)

tensor(0., device='cuda:0', dtype=torch.bfloat16, grad_fn=<DistBackward0>)

In [8]:
wgt_orig = mt._model.model.layers[5].mlp.up_proj.weight
wgt_finetuned = mt_check._model.model.layers[5].mlp.up_proj.weight

torch.dist(wgt_orig.cuda(), wgt_finetuned)

tensor(0.1875, device='cuda:0', dtype=torch.bfloat16, grad_fn=<DistBackward0>)